In [1]:
import numpy as np
from numpy import diff
from numpy import sum
from random import randint
import random
import math
import json

import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import trange

from models_params import Models_Params
import jsonpickle


np.__version__

2.2.0


'1.26.4'

In [2]:
class Yan:
    def __init__(self):
        # iniciar a coluna da desordem com -1 para indicar que as celulas estão vazias
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1, "x+": -1, "x-": -1, "y": -1}
        self.yangame = "YanGame"
        self.rolls_left = 2
        self.dices = [0,0,0,0,0]
        self.new_dice = list()
        self.marcado_em = ""
        self.over_minimum = False
        self.is_ended = False
        self.game_play = []
        self.next_state = self.get_game_state()
        self.reward = 0
        self.valid_moves_items = []
        

    def check_consecutive(self, l):
        n = len(l) - 1
        return sum(diff(sorted(l)) == 1) >= n


    def roll_dice(self, n):
        # random.seed(42)

        rolls = list()
        for i in range(n):
            rolls.append(randint(1, 6))
        rolls.sort()
        self.game_play.append("você rolou:                          " + str(rolls))
        return rolls


    def get_game_state(self):
        state = [self.rolls_left]
        for die in self.dices:
            state.append(die)
        for item in self.desordem:
            if self.desordem[item] == -1:
                state.append(1)
            else:
                state.append(0)
        return state
        

    def set_state(self, state):
        self.is_ended = False
        # state = ast.literal_eval(state)
        self.rolls_left = state[0]
        index = 1
        for die in range(len(self.dices)):
            self.dices[die] = state[index]
            index += 1

        index = 6
        for item in self.desordem:
            if state[index] == 1:
                self.desordem[item] = -1
            else:
                self.desordem[item] = 0
            index += 1

        return self.get_game_state()


    def get_empty_cells(self, state):
        self.set_state(state)
        avaiable_list = {}
        for item in self.desordem:
            if self.desordem[item] == -1:
                avaiable_list[item] = self.desordem[item]
        return avaiable_list
    

    def get_number_of_actions(self, state):
        return 13


    def do_the_reroll(self, n_dices):
        dices_to_reroll = '{0:05b}'.format(int(n_dices))
        self.game_play.append("dados a serem rolados novamente:     " + dices_to_reroll)
        for index in range(len(str(dices_to_reroll))):
            if dices_to_reroll[index] == "1":
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.game_play.append("seus dados ficaram assim:            " + str(self.dices))
        # dice_set = set(self.dices)
        # n_single = len(dice_set)
        # return (6 - n_single) * 3
        return 0


    def reset(self):
        self.game_play = []
        self.game_play.append("Jogo iniciado")
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1,
                         "x+": -1, "x-": -1,
                         "y": -1}
        self.dices = self.roll_dice(5)
        self.dices.sort()
        self.is_ended = False
        self.rolls_left = 2
        self.valid_moves_items = []
        # random.seed(42)

        initial_table = randint(1,8191)
        tabela = "{0:b}".format(initial_table)
        n_off_zeros = 13 - len(tabela)
        for i in range(n_off_zeros):
            tabela = "0" + tabela

        i = 0
        for item in self.desordem:
            if tabela[i] == "1":
                self.desordem[item] = -1
            else:
                self.desordem[item] = 0
            i += 1

        self.rolls_left = randint(0,2)

        return self.get_game_state()
    

    def get_initial_state(self):
        return self.reset()
    

    def get_valid_moves(self, state):
        self.valid_moves_items = []
        valid_action = []
        index = 0
        for item in self.desordem:
            if self.desordem[item] == -1:
                self.valid_moves_items.append(item)
                valid_action.append(1)
            else:
                valid_action.append(0)
            index += 1
        return valid_action


    def is_full(self):
        return ((self.dices[0] == self.dices[2] and self.dices[3] == self.dices[4]) or (
                        self.dices[0] == self.dices[1] and self.dices[2] == self.dices[4]))


    def set_cell_value(self, cell: str):

        self.is_ended = True
        self.game_play.append("você marcou                          " + str(self.dices) + " em " + cell)
        self.game_play.append("\n")
        
        # self.marcado_em = 
        # print(self.marcado_em)
        points = 0

        if cell == "1":
            points = self.dices.count(1)
            self.desordem["1"] = points
            return points
        
        elif cell == "2":
            points = self.dices.count(2) *2
            self.desordem["2"] = points
            return points
        
        elif cell == "3":
            points = self.dices.count(3) * 3
            self.desordem["3"] = points
            return points
        
        elif cell == "4":
            points = self.dices.count(4) * 4
            self.desordem["4"] = points
            return points
        
        elif cell == "5":
            points = self.dices.count(5) * 5
            self.desordem["5"] = points
            return points
        
        elif cell == "6":
            points = self.dices.count(6) * 6
            self.desordem["6"] = points
            return points
        
        elif cell == "y":
            if self.dices[0] == self.dices[4]:
                self.desordem["y"] = sum(self.dices) + 50
                return sum(self.dices) + 50
            else:
                self.desordem["y"] = 0
                return 0
            
        elif cell == "q":
            if self.dices[0] == self.dices[3]:
                self.desordem["q"] = sum(self.dices[:4]) + 30
                return sum(self.dices[:4]) + 30
            elif self.dices[1] == self.dices[4]:
                self.desordem["q"] = sum(self.dices[1:]) + 30
                return sum(self.dices[1:]) + 30
            else:
                self.desordem["q"] = 0
                return 0
            
        elif cell == "f":
            if self.is_full():
                self.desordem["f"] = sum(self.dices) + 20
                return sum(self.dices) + 20

            else:
                self.desordem["f"] = 0
                return 0
            
        elif cell == "s+":
            if self.check_consecutive(self.dices) and self.dices[0] == 2:
                self.desordem["s+"] = 60
                return 60
            else:
                self.desordem["s+"] = 0
                return 0
            
        elif cell == "s-":
            if self.check_consecutive( self.dices) and self.dices[0] == 1:
                self.desordem["s-"] = 50
                return 50
            else:
                self.desordem["s-"] = 0
                return 0
            
        elif cell == "x+":
            if (sum(self.dices) > self.desordem["x-"]) or self.desordem["x-"] == -1:
                self.desordem["x+"] = sum(self.dices)
                return sum(self.dices)
            else:
                self.desordem["x+"] = 0
            return 0
        
        elif cell == "x-":
            if (sum(self.dices) < self.desordem["x+"]) or self.desordem["x+"] == -1:
                self.desordem["x-"] = sum(self.dices)
                return sum(self.dices)
            else:
                self.desordem["x-"] = 0
            return 0
            

    def go_for_n(self, n):
        n_count = self.dices.count(n)
        if n_count == 5 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value(str(n))
            self.dices = self.roll_dice(5)
            self.dices.sort()
            if n_count > 2:
                reward *= 2
            return reward
        else:
            for index in range(5):
                if self.dices[index] != n:
                    self.dices[index] = self.roll_dice(1)[0]

        self.rolls_left -= 1
        self.dices.sort()
        improve = self.dices.count(n) - n_count
        return 0


    def go_for_y(self):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if np.max(dices_count) > 4 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("y")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward

        if dices_count.count(np.max(dices_count)) > 1:
            most_dices_indexs = []
            most_dices = None
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]
        else:        
            most_dices = np.argmax(dices_count) + 1

        original_most_count_die = self.dices.count(most_dices)

        for index in range(5):
            if self.dices[index] != most_dices:
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.rolls_left -= 1
        final_most_count_die = self.dices.count(most_dices)
        improve = final_most_count_die - original_most_count_die
        return 0


    def go_for_q(self):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if np.max(dices_count) > 3 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("q")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward

        if dices_count.count(np.max(dices_count)) > 1:
            most_dices_indexs = []
            most_dices = None
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]
        else:        
            most_dices = np.argmax(dices_count) + 1

        original_most_count_die = self.dices.count(most_dices)

        for index in range(5):
            if self.dices[index] != most_dices:
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.rolls_left -= 1
        final_most_count_die = self.dices.count(most_dices)
        improve = final_most_count_die - original_most_count_die
        return 0


    def get_dices_for_f(self):
        dices_count = []
        least_dices = None
        most_dices = None


        for n in range (1,7):
            dices_count.append(self.dices.count(n))

        if dices_count.count(np.max(dices_count)) == 2:
            for index in range(len(dices_count)):
                if dices_count[index] == 1:
                    least_dices = (index + 1)

        elif dices_count.count(np.max(dices_count)) == 1:
            most_dices = np.argmax(dices_count) +1

        elif dices_count.count(np.max(dices_count)) > 2:
            most_dices = None
            most_dices_indexs = []
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]

        return least_dices, most_dices


    def go_for_f(self):
        
        if self.is_full() or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("f")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        initial_least_dices, initial_most_dices = self.get_dices_for_f()

        initial_count = 0
        final_count = 0

        if initial_least_dices != None:
            initial_count = 1
            for index in range(5):
                if self.dices[index] == initial_least_dices:
                    self.dices[index] = self.roll_dice(1)[0]
            
        if initial_most_dices != None:
            initial_count = self.dices.count(initial_most_dices)
            for index in range(5):
                if self.dices[index] != initial_most_dices:
                    self.dices[index] = self.roll_dice(1)[0]
            
        final_least_dices, final_most_dices = self.get_dices_for_f()
        if initial_least_dices != None:
            if final_least_dices == None:
                final_count = 1
            else:
                final_count = 0
        else:
            if final_least_dices != None:
                final_count = 2 + (2 - initial_count)
            else:
                final_count = self.dices.count(final_most_dices) - initial_count

        self.dices.sort()
        self.rolls_left -= 1
        return 0


    def get_reroll_dices_s(self, witch_s):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
            
        reroll_dices = []

        if witch_s == "s+":
            for n in range(dices_count[0]):
                reroll_dices.append(1)
                
            for index in range(1, 6):
                for quant in range(dices_count[index] - 1):
                    reroll_dices.append(index + 1)

        else:
            for n in range(dices_count[5]):
                reroll_dices.append(6)

            for index in range(5):
                for quant in range(dices_count[index] - 1):
                    reroll_dices.append(index + 1)
       
        reroll_dices.sort()

        return reroll_dices


    def go_for_s(self, witch_s):
        
        if self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value(witch_s)
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        if self.check_consecutive(self.dices) and self.dices[0] == 2 and witch_s == "s+":
            self.rolls_left = 2
            reward = self.set_cell_value("s+")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        if self.check_consecutive(self.dices) and self.dices[0] == 1 and witch_s == "s-":
            self.rolls_left = 2
            reward = self.set_cell_value("s-")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        reroll_dices = self.get_reroll_dices_s(witch_s)
        original_n_dice_roled = len(reroll_dices)

        for index in range(5):
            if self.dices[index] in reroll_dices:
                reroll_dices = np.delete(reroll_dices, 0)
                self.dices[index] = self.roll_dice(1)[0]

        self.rolls_left -= 1
        self.dices.sort()
        final_n_dice_roled = len(self.get_reroll_dices_s(witch_s))
        improve = original_n_dice_roled - final_n_dice_roled
        return 0
    

    def get_next_state(self, state, action):
        if action == None:
            return (self.get_game_state(), 0, self.is_ended)
        self.set_state(state)
        action = int(action)
        self.reward = 0

        if action < 6:
            self.reward = self.go_for_n(action+1)/20
        elif action == 6:
            self.reward = self.go_for_q()*10
        elif action == 7:
            self.reward = self.go_for_f()*10
        elif action == 8:
            self.reward = self.go_for_s("s+")*10
        elif action == 9:
            self.reward = self.go_for_s("s-")*10
        elif action == 10:
            self.rolls_left = 2
            self.reward = self.set_cell_value("x+")
            self.reward = 0
            self.dices = self.roll_dice(5)
            self.dices.sort()
        elif action == 11:
            self.rolls_left = 2
            self.reward = self.set_cell_value("x-")
            self.reward = 0
            self.dices = self.roll_dice(5)
            self.dices.sort()
        elif action == 12:
            self.reward = self.go_for_y()*10
        else:
            breakpoint()
            print(action)
        return (self.get_game_state(), self.reward, self.is_ended)


    # def check_ended(self, state):
    #     self.set_state(state)
    #     return -1 not in self.desordem.values()
    

    def get_value_and_terminated(self, state, action):
        if action == None:
            return (0, False)
        self.set_state(state)
        next_state, reward, is_ended = self.get_next_state(state, action)
        return reward, is_ended
    

    def get_total_score(self, state):
        self.set_state(state)
        self.score_values = list(self.desordem.values())
        self.total = sum(self.score_values)
        total_upper = sum(self.score_values[0:6])
        if total_upper >= 60:
            self.total += 30
        # print()
        self.game_play.append("sua tabela ficou assim:              " + str(self.desordem))
        self.game_play.append("TOTAL:                               " + str(self.total))
        # print("total: ", self.total)
        # print("sua tabela ficou assim: ", self.desordem)
        # print()
        

        return self.total
    

    def get_encoded_state(self, state):

        encoded_state = []
        encoded_state.append(state[0]/2)
        for i in range(1,6):
            encoded_state.append(state[i]/6)
        encoded_state += state[6:]
        return encoded_state
    

In [3]:
NUM_CLASSES = 13
NUM_FEATURES = 3
RANDOM_SEED = 42


In [4]:
class ResNet(nn.Module):
    def __init__(self, game, num_resBlocks, num_hidden):
        super().__init__()
        self.startBlock = nn.Sequential(
            nn.Linear(in_features=19, out_features=num_hidden),
            nn.ReLU()
        )
        
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden) for i in range(num_resBlocks)]
        )
        
        self.policyHead = nn.Sequential(
            nn.Linear(in_features=num_hidden, out_features=num_hidden),
            # nn.BatchNorm1d(num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 13)
        )
        
        self.valueHead = nn.Sequential(
            nn.Linear(in_features=num_hidden, out_features=num_hidden),
            # nn.BatchNorm1d(num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 1),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
        
        
class ResBlock(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        self.lin1 = nn.Linear(in_features=num_hidden, out_features=num_hidden)
        # self.bn1 = nn.BatchNorm1d(num_hidden)
        self.lin2 = nn.Linear(in_features=num_hidden, out_features=num_hidden)
        # self.bn2 = nn.BatchNorm1d(num_hidden)
        
    def forward(self, x):
        residual = x
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x += residual
        x = F.relu(x)
        return x
        

In [5]:
class Node:
    def __init__(self, game:Yan, args, state, parent=None, action_taken=None, prior=0):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior
        
        self.children = []
        
        self.visit_count = 0
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        if child.visit_count == 0:
            q_value = 0
        else:
            q_value = ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * (math.sqrt(self.visit_count) / (child.visit_count + 1)) * child.prior
    
    def expand(self, policy):
        child = None
        for action, prob in enumerate(policy):
            if prob > 0:
                child_state = self.state.copy()
                child_state, child_reward, child_terminated = self.game.get_next_state(child_state, action)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
        if child == None:
            breakpoint()
        return child
    
            
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        # value = self.game.reward
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game:Yan, args, model):
        self.game = game
        self.args = args
        self.model = model

    @torch.no_grad() 
    def search(self, state):
        root = Node(self.game, self.args, state)
        
        for search in range(self.args['num_searches']):
            node = root
            
            while node.is_fully_expanded():
                node = node.select()
                
            state, value, is_terminal = self.game.get_next_state(node.state, node.action_taken)
            
            if not is_terminal:
                tensor = torch.tensor(self.game.get_encoded_state(node.state)).unsqueeze(0)
                policy_init, value = self.model(
                    tensor.float()
                )   
                
                if np.isnan(policy_init[0][0]):
                    breakpoint()
                policy = torch.softmax(policy_init, axis=1).squeeze(0).cpu().numpy()
                valid_moves = self.game.get_valid_moves(node.state)
                policy *= valid_moves
                sum_policy = np.sum(policy)
                if sum_policy != 0:
                    policy = policy/sum_policy
                    node.expand(policy)

                # if np.isnan(out_policy[0]):
                #     out_policy = torch.softmax(policy_init, axis=1).squeeze(0).cpu().numpy()
                #     out_policy *= valid_moves
                # else:
                #     breakpoint()
                value = value.item()
                

            node.backpropagate(value)    
            
            
        # action_probs = self.game.get_valid_moves(state)
        action_probs = np.zeros(13)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        if np.sum(action_probs) != 0:
            action_probs /= np.sum(action_probs)
        if np.isnan(action_probs[0]):
            breakpoint()
        return action_probs
        
        
        

In [6]:
class AlphaZero:
    def __init__(self, model, optimizer, game:Yan, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTS(game, args, model)
        
    def selfPlay(self):
        memory = []
        # player = 1
        state = self.game.get_initial_state()
        
        while True:
            # neutral_state = self.game.change_perspective(state, player)
            action_probs = self.mcts.search(state)
            
            memory.append((state, action_probs))

            if np.sum(action_probs) == 0:
                breakpoint()
                returnMemory = [(self.game.get_encoded_state(state), action_probs, 0)]
                return returnMemory
            
            action = np.random.choice(self.game.get_number_of_actions(state), p=action_probs)
            
            state, reward, terminal = self.game.get_next_state(state, action)
            
            value, is_terminal = self.game.get_value_and_terminated(state, action)
            
            if is_terminal:
                returnMemory = []
                for hist_state, hist_action_probs in memory:
                    hist_outcome = value
                    returnMemory.append((
                        self.game.get_encoded_state(hist_state),
                        hist_action_probs,
                        hist_outcome
                    ))
                return returnMemory
            
            # player = self.game.get_opponent(player)
                
    def train(self, memory):
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])] 
            try:
                state, policy_targets, value_targets = zip(*sample)
                
                state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
                
                state = torch.tensor(state, dtype=torch.float32)
                policy_targets = torch.tensor(policy_targets, dtype=torch.float32)
                value_targets = torch.tensor(value_targets, dtype=torch.float32)
                
                out_policy, out_value = self.model(state)
                
                policy_loss = F.cross_entropy(out_policy, policy_targets)
                value_loss = F.mse_loss(out_value, value_targets)
                loss = policy_loss + value_loss
                
                self.optimizer.zero_grad() # change to self.optimizer
                loss.backward()
                self.optimizer.step() # change to self.optimizer
            except:
                pass
    
    def learn(self):
        for iteration in trange(self.args['num_iterations']):
            memory = []
            
            self.model.eval()
            for selfPlay_iteration in range(self.args['num_selfPlay_iterations']):
                memory += self.selfPlay()
                
            self.model.train()
            for epoch in range(self.args['num_epochs']):
                self.train(memory)
            version = self.args['model_version']
            if (iteration + 1) % 10 == 0:
                torch.save(self.model.state_dict(), f"models/model_{version}_{iteration}.pt")
                torch.save(self.optimizer.state_dict(), f"models/optimizer_{version}_{iteration}.pt")

In [7]:
# yan = Yan()
# init_state = yan.get_initial_state()
# print(init_state)
# encoded_state = yan.get_encoded_state(init_state)
# print(encoded_state)
# dados = encoded_state[1]*66666
# print(int(dados))
# tabela = int(encoded_state[2]*8191)
# print(tabela)

# tabela = "{0:b}".format(tabela)
# print(tabela)


In [8]:
# initial_table = randint(0,8191)
# print(initial_table)
# tabela = "{0:b}".format(initial_table)
# n_off_zeros = 13 - len(tabela)
# for i in range(n_off_zeros):
#     tabela = "0" + tabela

# tabela_array = []
# for i in range(13):
#     tabela_array.append(int(tabela[i]))
# print(tabela_array)

# initial_reroll = randint(0,2)
# print(initial_reroll)

# initial_dice = []
# for i in range(5):
#     initial_dice.append(randint(1,6))
# print(initial_dice)

# initial_random_state = []
# initial_random_state.append(initial_reroll)
# initial_random_state += initial_dice
# initial_random_state += tabela_array
# print(initial_random_state)

# encoded_state = []
# encoded_state.append(initial_random_state[0]/2)
# for i in range(1,6):
#     encoded_state.append(initial_random_state[i]/6)
# encoded_state += initial_random_state[6:]
# print(encoded_state)


In [9]:
game_params = {
    "go_for_n_factor": 0.05,
    "go_for_q_factor": 10,
    "go_for_f_factor": 10,
    "go_for_s_factor": 10,
    "go_for_s__factor": 10,
    "go_for_y_factor": 10,
    "go_for_x_factor": 0,
    "go_for_x__factor": 0
}

In [10]:
model_version = "v0"


In [15]:
filename = "modelsparams.json"
game = Yan()
saved_models_param_dict = {}
with open(filename) as json_file:
        jsonObj = json.load(json_file)
        unpickled = jsonpickle.decode(jsonObj, classes=Models_Params)
saved_models_param_dict = unpickled

model = ResNet(game, 4, 62)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

args = {
    
    'C': 2,
    'num_searches': 220,
    'num_iterations': 100,
    'num_selfPlay_iterations': 200,
    'num_epochs': 100,
    'batch_size': 128,
    'model_version': model_version
}
saved_params = Models_Params(
    args=args, 
    version=model_version, 
    policy_loss="cross_entropy", 
    value_loss="mse_loss",
    optim="Adam",
    lr=0.0001,
    game_params=game_params
    )

saved_models_param_dict[model_version] = saved_params

with open(filename, "w") as outfile:
    json.dump(jsonpickle.encode(saved_models_param_dict), outfile)


alphaZero = AlphaZero(model, optimizer, game, args)
alphaZero.learn()

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
game = Yan()
args = {
    'C': 2,
    'num_searches': 10000
}
# model = BobNet()
model = ResNet(game, 4, 62)
model.load_state_dict(torch.load(f"models/model_{model_version}_99.pt"))

model.eval()
mcts = MCTS(game, args, model)


def play_game():
    state = game.get_initial_state()

    total_reward = 0
    while True:
        game.get_valid_moves(state)
        print(state)
        print(f"rerolls:        {state[0]}")
        print(f"dice:           {state[1:6]}")
        print(f"table:          {game.valid_moves_items}")
        mcts_probs = mcts.search(state)
        action = np.argmax(mcts_probs)
        print(f"action:         {list(game.desordem.keys())[action]}")
        state, reward, is_terminal = game.get_next_state(state, action)
        print(f"reward:         {reward}")
        total_reward += reward
        print(f"total_reward:   {total_reward}\n")
        if is_terminal:
            print(game.desordem)
            print(game.get_total_score(state))
            game.reset()
            break
            

play_game()

In [ ]:
import matplotlib.pyplot as plt

yan = Yan()

state = yan.get_initial_state()

# state = yan.get_next_state(state, 2, 1)
# state = yan.get_next_state(state, 7, -1)

# print(state)

encoded_state = yan.get_encoded_state(state)
yan.get_valid_moves(state)
print(state)
print(f"rerolls:        {state[0]}")
print(f"dice:           {state[1:6]}")
print(f"table:          {yan.valid_moves_items}")
# print(f"action:         {list(game.desordem.keys())[action]}")
# state, reward, is_terminal = game.get_next_state(state, action)
# print(f"reward:         {reward}")
        

tensor_state = torch.tensor(encoded_state).unsqueeze(0)
# model = BobNet().to(device)
model = ResNet(yan, 4, 62)

model.load_state_dict(torch.load(f"models/model_{model_version}_99.pt"))
model.eval()
with torch.inference_mode():
    policy, value = model(tensor_state)
value = value.item()
policy = torch.softmax(policy, axis=1)

# print(value)
# for action in policy[0]:
#     print(action)

plt.bar(range(13), policy.squeeze(0).numpy())  
plt.show()


In [ ]:
from playsound import playsound

In [ ]:
playsound("done.mp3")